In [1]:
import torch as t
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset

In [ ]:
class ResBlock(nn.Module):
    def __init__(self, input_channels) -> None: #input image should be a 64x64 I THINK IM NOT SURE
        super(ResBlock, self).__init__()
        self.nonlinearity = nn.ReLU()
        self.padding = nn.ReflectionPad2d(1)
        self.convolution = nn.Conv2d(input_channels, input_channels, 3)#(64 + 2)/3 + 1 = 23
        self.normalization = nn.InstanceNorm2d(input_channels)
    
    def forward(self, x):
        firstPadding = self.padding(x)
        firstConv = self.convolution(firstPadding)
        firstNormalization = self.normalization(firstConv)
        firstNonLinearity = self.nonlinearity(firstNormalization)

        secondPadding = self.padding(firstNonLinearity)
        secondConv = self.convolution(secondPadding)
        secondNormalization = self.normalization(secondConv)
        finalNonLinearity = self.nonlinearity(secondNormalization)
        return finalNonLinearity + x

         